In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import datetime
import copy
import json

import sys
sys.path.append('../../')

from data.dataloader import Covid19IndiaLoader
from data.processing import get_data

from models.seir import SEIRHD, SEIR_Testing, SEIR_Movement, SEIR_Movement_Testing

from main.seir.fitting import single_fitting_cycle, get_variable_param_ranges
from main.seir.uncertainty import MCUncertainty
from main.seir.forecast import get_forecast, create_region_csv, create_all_csvs, write_csv, get_all_trials
from main.seir.sensitivity import sensitivity_all_comps

from utils.create_report import create_report, trials_to_df
from utils.loss import Loss_Calculator
from utils.enums import Columns

from viz import plot_forecast, plot_fit, plot_fit_multiple_preds

## Load Covid19india Data

In [ ]:
loader = Covid19IndiaLoader()
dataframes = loader.get_covid19india_api_data()

In [ ]:
predictions_dict = {}

## Select Districts to fit on

In [ ]:
districts_to_show = [('Maharashtra', 'Mumbai')]

## Params

In [ ]:
date_of_interest = '2020-07-31'
forecast_days = 37

## Perform M1 and M2 fits

In [ ]:
for state, district in districts_to_show:
    predictions_dict[(state, district)] = {}
    predictions_dict[(state, district)]['m1'] = single_fitting_cycle(
        dataframes, state, district, data_from_tracker=True, granular_data=False, filename=None, #Data
        model=SEIRHD, #Choose Model and Ranges
        train_period=7, val_period=7, num_evals=1000, initialisation='intermediate', #Optimisation related parameters
        which_compartments=['total_infected', 'hospitalised', 'recovered', 'deceased'], #Compartments to Apply Loss on 
        smooth_jump=True)
    
    predictions, losses, params = get_all_trials(predictions_dict[(state, district)], train_fit='m1', forecast_days=forecast_days)
    predictions_dict[(state, district)]['m1']['params'] = params
    predictions_dict[(state, district)]['m1']['losses'] = losses
    predictions_dict[(state, district)]['m1']['predictions'] = predictions
    predictions_dict[(state, district)]['m1']['all_trials'] = trials_to_df(predictions, losses, params, column=Columns.active)
    
    predictions_dict[(state, district)]['m2'] = single_fitting_cycle(
        dataframes, state, district, data_from_tracker=True, granular_data=False, filename=None, #Data
        model=SEIRHD, #Choose Model and Ranges
        train_period=7, val_period=0, num_evals=1000, initialisation='intermediate', #Optimisation related parameters
        which_compartments=['total_infected', 'hospitalised', 'recovered', 'deceased'], #Compartments to Apply Loss on 
        smooth_jump=True)
    
    predictions, losses, params = get_all_trials(predictions_dict[(state, district)], train_fit='m2', forecast_days=forecast_days)
    predictions_dict[(state, district)]['m2']['params'] = params
    predictions_dict[(state, district)]['m2']['losses'] = losses
    predictions_dict[(state, district)]['m2']['predictions'] = predictions
    predictions_dict[(state, district)]['m2']['all_trials'] = trials_to_df(predictions, losses, params, column=Columns.active)
    
    predictions_dict[(state, district)]['state'] = state
    predictions_dict[(state, district)]['dist'] = district
    predictions_dict[(state, district)]['fitting_date'] = datetime.datetime.now().strftime("%Y-%m-%d")
    predictions_dict[(state, district)]['datasource'] = 'covid19api' if predictions_dict[(state, district)]['m1']['data_from_tracker'] else 'municipality'
    predictions_dict[(state, district)]['variable_param_ranges'] = predictions_dict[(state, district)]['m1']['variable_param_ranges']
    predictions_dict[(state, district)]['data_last_date'] = predictions_dict[(state, district)]['m2']['data_last_date']

## Loss DataFrames

### M1 Loss DataFrame

In [ ]:
lc = Loss_Calculator()

In [ ]:
lc.create_loss_dataframe_master(predictions_dict, 'm1')

### M2 Loss DataFrame

In [ ]:
lc.create_loss_dataframe_master(predictions_dict, 'm2')

## Uncertainty

In [ ]:
region_dict = predictions_dict[('Maharashtra', 'Mumbai')]
uncertainty = MCUncertainty(region_dict, date_of_interest)

In [ ]:
uncertainty.beta

In [ ]:
deciles_forecast = uncertainty.get_forecasts()
df_prediction_decile50 = deciles_forecast[50]['df_prediction']
df_prediction_decile50 = df_prediction_decile50[df_prediction_decile50['date'].isin(predictions_dict[('Maharashtra', 'Mumbai')]['m1']['df_val']['date'])]
predictions_dict[('Maharashtra', 'Mumbai')]['m1']['df_prediction_decile50'] = df_prediction_decile50

In [ ]:
plot_fit_multiple_preds(predictions_dict[('Maharashtra', 'Mumbai')], which_fit='m1')

## Plot Forecasts

In [ ]:
for region in predictions_dict.keys():
    predictions_dict[region]['forecast'] = plot_forecast(predictions_dict[region], region, both_forecasts=True, error_bars=True)

## Create Report

In [ ]:
for region in predictions_dict.keys():
    create_report(predictions_dict[region])

## Create and Save Output CSV

In [ ]:
df_output = create_all_csvs(predictions_dict, icu_fraction=0.02)

In [ ]:
write_csv(df_output)